In [3]:
# #First approach to fix the issue: implement a smooth raster method by applying a convolution filter, such as Gaussian blur. 
# # This operation will average each pixel with its neighbors, whighted by a kernel, reducing high-frequency noise

# import osmnx as ox
# import geopandas as gpd
# import pandas as pd
# import numpy as np
# import rasterio
# from shapely.geometry import LineString
# from pyproj import CRS, Transformer
# from rasterio.windows import Window
# from geopy.distance import geodesic
# from scipy.ndimage import gaussian_filter
# from rasterio import MemoryFile

# # Get the network
# G = ox.graph_from_place('Montereale Valcellina, Italy', network_type='all')
# edges = ox.graph_to_gdfs(G, nodes=False)

# # Transform the entire GeoDataFrame to the same CRS as the DEM
# edges = edges.to_crs(CRS('EPSG:32632'))

# # Filter the major roads
# major_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
#                'tertiary_link', 'trunk', 'trunk_link', 'residential', 'cycleway',
#                'living_street', 'unclassified', 'motorway', 'motorway_link',
#                'pedestrian', 'steps', 'track']
# edges = edges[edges['highway'].isin(major_roads)]

# # Set the path to the DEM file
# dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/w51075_s10.tif'

# # Load the DEM and apply a Gaussian filter
# with rasterio.open(dem_path) as src:
#     dem_data = src.read(1)  # Read the DEM data into a numpy array
#     dem_profile = src.profile  # Save the profile attribute to a variable

# # Apply the Gaussian filter
# sigma = 1  # You may need to adjust this value
# smoothed_dem_data = gaussian_filter(dem_data, sigma)

# # Create a new rasterio dataset with the smoothed DEM data
# with MemoryFile() as memfile:
#     with memfile.open(**dem_profile) as dest:  # Use the saved profile
#         dest.write(smoothed_dem_data, 1)

#     # Now you can use the smoothed DEM in the same way as the original DEM
#     dem = memfile.open()


# # Calculate the slope
# def calc_slope(row):
#     coords = row['geometry'].coords.xy
#     elevation = []

#     for i in range(len(coords[0])-1):
#         start = (coords[0][i], coords[1][i])
#         end = (coords[0][i+1], coords[1][i+1])

#         start_elevation = list(dem.sample([start]))[0][0]
#         end_elevation = list(dem.sample([end]))[0][0]
#         dz = end_elevation - start_elevation

#         dx = end[0] - start[0]
#         dy = end[1] - start[1]
#         distance = np.sqrt(dx ** 2 + dy ** 2)

#         slope = (dz / distance) * 100 if distance != 0 else 0
#         elevation.append(slope)

#     return np.mean(elevation)

# # Apply function to each row in the GeoDataFrame
# edges['slope'] = edges.apply(calc_slope, axis=1)

# # Replace NaN slope values with 0
# edges['slope'] = edges['slope'].fillna(0)

# # Classify slopes
# edges['slope_class'] = pd.cut(edges['slope'],
#                               bins=[-30, 3, 5, 8, 10, 20, np.inf],
#                               labels=["0-3: flat", "3-5: mild", "5-8: medium",
#                                       "8-10: hard", "10-20: extreme", ">20: impossible"],
#                               right=False)
# edges['slope_class'] = edges['slope_class'].fillna("0-3: flat")

# # Calculate the proportion of each slope class
# slope_class_distribution = round(edges['slope_class'].value_counts(normalize=True) * 100, 1)
# print(slope_class_distribution)


0-3: flat          91.2
3-5: mild           3.1
5-8: medium         2.4
10-20: extreme      1.7
8-10: hard          1.5
>20: impossible     0.2
Name: slope_class, dtype: float64
